In [1]:
import json,os,shutil,yaml,torch
import torch.nn as nn


from models import create_model
from utils.train import train_model, test_model
from utils.util import get_optimizer, get_loss, get_scheduler
from utils.data_container import get_data_loader
from utils.preprocess import  preprocessing_for_metric



def train(conf, data_category):
    print(json.dumps(conf, indent=4))

    #os.environ["CUDA_VISIBLE_DEVICES"] = str(conf['device'])
    os.environ["CUDA_VISIBLE_DEVICES"] = str(1)
    device = torch.device(0)

    model_name = conf['model']['name']
    optimizer_name = conf['optimizer']['name']
    data_set = conf['data']['dataset']
    scheduler_name = conf['scheduler']['name']
    loss = get_loss(**conf['loss'])

    loss.to(device)


    support = preprocessing_for_metric(data_category=data_category, dataset=conf['data']['dataset'],
                                           Normal_Method=conf['data']['Normal_Method'], _len=conf['data']['_len'], **conf['preprocess'])
    model, trainer = create_model(model_name,
                                  loss,
                                  conf['model'][model_name],
                                  data_category,
                                  device,
                                  support)

    optimizer = get_optimizer(optimizer_name, model.parameters(), conf['optimizer'][optimizer_name]['lr'])
    scheduler = get_scheduler(scheduler_name, optimizer, **conf['scheduler'][scheduler_name])
    if torch.cuda.device_count() > 1:
        print("use ", torch.cuda.device_count(), "GPUS")
        model = nn.DataParallel(model)
    else:
        model.to(device)

    save_folder = os.path.join('save', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])
    run_folder = os.path.join('run', conf['name'], f'{data_set}_{"".join(data_category)}', conf['tag'])

    shutil.rmtree(save_folder, ignore_errors=True)
    os.makedirs(save_folder)
    shutil.rmtree(run_folder, ignore_errors=True)
    os.makedirs(run_folder)

    with open(os.path.join(save_folder, 'config.yaml'), 'w+') as _f:
        yaml.safe_dump(conf, _f)

    data_loader, normal = get_data_loader(**conf['data'], data_category=data_category, device=device,
                                          model_name=model_name)


    train_model(model=model,
                       dataloaders=data_loader,
                       trainer=trainer,
                       optimizer=optimizer,
                       normal=normal,
                       scheduler=scheduler,
                       folder=save_folder,
                       tensorboard_folder=run_folder,
                       device=device,
                       **conf['train'])
    test_model(folder=save_folder,
                      trainer=trainer,
                      model=model,
                      normal=normal,
                      dataloaders=data_loader,
                      device=device)


if __name__ == '__main__':
    # parser = argparse.ArgumentParser()
    # parser.add_argument('--config', required=True, type=str, default='dsgrn-config',
    #                     help='Configuration filename for restoring the model.')
    # parser.add_argument('--con', required=False, type=str2bool, default='False',
    #                     help='Test.')
    # parser.add_argument('--stage', required=True, type=str2bool, default='True',
    #                     help='Stage.')
    # args = parser.parse_args()
    con = 'evoconv2-config'
    data = ['bike']
    with open(os.path.join('config', f'{con}.yaml')) as f:
        conf = yaml.safe_load(f)
    train(conf, data)


{
    "name": "Evoconv2",
    "tag": "train1",
    "device": 0,
    "data": {
        "dataset": "nogrid",
        "batch_size": 32,
        "X_list": [
            12,
            11,
            10,
            9,
            8,
            7,
            6,
            5,
            4,
            3,
            2,
            1
        ],
        "Y_list": [
            0,
            1,
            2,
            3,
            4,
            5,
            6,
            7,
            8,
            9,
            10,
            11
        ],
        "_len": [
            672,
            672
        ],
        "Normal_Method": "Standard"
    },
    "preprocess": {
        "hidden_size": 20,
        "normalized_category": "randomwalk"
    },
    "train": {
        "epochs": 200,
        "max_grad_norm": 5,
        "early_stop_steps": 10
    },
    "optimizer": {
        "name": "Adam",
        "Adam": {
            "lr": 0.0005,
            "weight_decay": 0.0005,
            

train epoch:   0, train loss: 4.03245: : 94it [00:18,  5.17it/s]
validate epoch:   0, validate loss: 4.84322: : 21it [00:00, 24.38it/s]


Better model at epoch 0 recorded.


test  epoch:   0, test  loss: 5.05964: : 21it [00:00, 25.09it/s]
train epoch:   1, train loss: 3.18362: : 94it [00:18,  5.09it/s]
validate epoch:   1, validate loss: 4.50127: : 21it [00:00, 24.22it/s]


Better model at epoch 1 recorded.


...

Better model at epoch 29 recorded.


test  epoch:  29, test  loss: 2.84966: : 21it [00:00, 24.08it/s]
train epoch:  30, train loss: 2.30469: : 94it [00:18,  5.09it/s]
validate epoch:  30, validate loss: 2.81835: : 21it [00:00, 23.62it/s]
test  epoch:  30, test  loss: 2.96549: : 21it [00:00, 23.15it/s]
train epoch:  31, train loss: 2.30248: : 94it [00:18,  5.09it/s]
validate epoch:  31, validate loss: 2.75764: : 21it [00:00, 23.51it/s]
test  epoch:  31, test  loss: 2.88399: : 21it [00:00, 23.83it/s]
train epoch:  32, train loss: 2.30587: : 94it [00:18,  5.07it/s]
validate epoch:  32, validate loss: 2.78812: : 21it [00:00, 23.52it/s]
test  epoch:  32, test  loss: 2.91382: : 21it [00:00, 23.58it/s]
train epoch:  33, train loss: 2.29851: : 94it [00:18,  5.09it/s]
validate epoch:  33, validate loss: 2.76151: : 21it [00:00, 23.99it/s]
test  epoch:  33, test  loss: 2.88916: : 21it [00:00, 23.69it/s]
train epoch:  34, train loss: 2.30471: : 94it [00:18,  5.12it/s]
validate epoch:  34, validate loss: 2.76932: : 21it [00:00, 23.86i

cost 925.2539465436712 seconds
model of epoch 29 successfully saved at `save/Evoconv2/nogrid_bike/train1/best_model.pkl`


21it [00:00, 26.11it/s]


test results:
{
    "MAE": {
        "horizon-0": 1.6422478423134004,
        "horizon-1": 1.6604370818195249,
        "horizon-2": 1.6855596587204797,
        "horizon-3": 1.705145652646017,
        "horizon-4": 1.729362078095959,
        "horizon-5": 1.7614194265538932,
        "horizon-6": 1.7965538894155764,
        "horizon-7": 1.8199155542248375,
        "horizon-8": 1.8344108700897193,
        "horizon-9": 1.8426612463911918,
        "horizon-10": 1.8549867944384613,
        "horizon-11": 1.8767794858569422
    },
    "RMSE": {
        "horizon-0": 2.5946983499756686,
        "horizon-1": 2.63771437043956,
        "horizon-2": 2.702657295996356,
        "horizon-3": 2.7616343460411956,
        "horizon-4": 2.8180497434612652,
        "horizon-5": 2.8852206399162124,
        "horizon-6": 2.958441310368821,
        "horizon-7": 2.9975772051391507,
        "horizon-8": 3.023009919604782,
        "horizon-9": 3.035125960464815,
        "horizon-10": 3.05490210038858,
        "horizo